In [1]:
#This is transfer learning
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape = (300, 300, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(r'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')#loading the weight

for layer in pre_trained_model.layers:
  layer.trainable = False  #Making the layer not to train again
  
pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')  #loading layer required
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [2]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (2, activation='softmax')(x)           

#the trained layer has 2048 classes but we are reducing it to 2 classes(mask and non mask)
model = Model( pre_trained_model.input, x) 
model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [3]:
import cv2,os

data_path=r'trainset'#r for reading
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary
#labelling it
print(label_dict)
print(categories)
print(labels)

['nmask.jpg', 'nmask1.jpg', 'nmask10.jpg', 'nmask11.jpg', 'nmask12.jpg', 'nmask13.jpg', 'nmask14.jpg', 'nmask15.jpg', 'nmask16.jpg', 'nmask17.jpg']
['mask.jpg', 'mask1.jpg', 'mask10.jpg', 'mask11.jpg', 'mask12.jpg', 'mask13.jpg', 'mask14.jpg', 'mask15.jpg', 'mask16.jpg', 'mask17.jpg']


In [4]:
img_size=300
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            #gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(img,(img_size,img_size))
            
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image


total training nmask images: 100
total training mask images: 100


In [5]:
import numpy as np
data=np.array(data)
data=data/255
data=np.reshape(data,(data.shape[0],img_size,img_size,3 ))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

from sklearn.utils import shuffle
data,Label = shuffle(data,new_target, random_state=2)
train_data = [data,Label]


In [6]:
history=model.fit(train_data[0],train_data[1],epochs=10)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [7]:
import cv2
import numpy as np

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#source=cv2.VideoCapture(r"C:\Users\Acer\Desktop\face mask detector\face-mask-detection-keras-master\test_video.mp4")
source=cv2.VideoCapture(0)
#source=cv2.imread(r"C:\Users\Acer\Desktop\face mask detector\face-mask-detection-keras-master\group_mask2")

source.set(1,320)
source.set(2,480)


labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [8]:
while(True):

    ret,img=source.read()
    #gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(img,1.4,5)  

    for (x,y,w,h) in faces:
    
        face_img=img[y:y+w,x:x+w]
        a=int(y)
        b=int(x)
        a=a/2
        b=b/2
        print(a)
        resized=cv2.resize(face_img,(300,300))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,300,300,3))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

Found 200 images belonging to 2 classes.
Found 30 images belonging to 2 classes.
